In [2]:
import polars as pl
from datetime import datetime

In [3]:
df = pl.read_json('../dataset/trans.json')

In [4]:
# Split into chunks manually
chunk_size = 2
chunks = [df[i:i + chunk_size] for i in range(0, 1000, chunk_size)]

# Process each chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:")
    print(chunk)  # Print the chunk

Chunk 1:
shape: (2, 166)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ vendorAcc ┆ workedOnB ┆ fraudBill ┆ postedToH ┆ … ┆ reversalP ┆ createdBy ┆ oltpvFine ┆ pushedUw │
│ ountName  ┆ y         ┆ Id        ┆ ubtelDate ┆   ┆ urpose    ┆ Name      ┆ ractSavin ┆ pToPolym │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ gsAccount ┆ orphOn   │
│ str       ┆ null      ┆ null      ┆ str       ┆   ┆ null      ┆ null      ┆ Id        ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ ---       ┆ null     │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ str       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ MTN Ghana ┆ null      ┆ null      ┆ 2024-11-1 ┆ … ┆ null      ┆ null      ┆ 2261480   ┆ null     │
│           ┆           ┆           ┆ 3T19:55:0 ┆   ┆           ┆ 

In [5]:
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [6]:
df.shape

(133949, 166)

In [7]:
df['status'].unique

<bound method Series.unique of shape: (133_949,)
Series: 'status' [str]
[
	"success"
	"success"
	"success"
	"success"
	"success"
	…
	"unpaid"
	"success"
	"paid"
	"paid"
	"success"
]>

In [8]:
data = df.select(
    [
        'accountNumber', 'amount', 'amountAfter', 'amountRefunded', 
        'createdAt', 'month', 'meterCategory', 'meterDistrict', 
        'meterRegion', 'paymentChannel', 'paymentProvider', 
        'rechargeSource', 'status'
    ]
).filter(pl.col('status') == 'success')

#sum of the entire dataset / Total Revenue
data['amount'].sum()

7047202

In [12]:
#average of the entire dataset
data['amount'].mean()

80.94556690136801

In [17]:
df.select(pl.std('amount'))

amount
f64
117.791306


In [18]:
#average revenue per customer 
data.group_by(pl.col('accountNumber')).agg(pl.col('amount').mean())

accountNumber,amount
str,f64
"""733070170""",20.0
"""711180602""",20.0
"""747880581""",5.0
"""748589863""",94.0
"""708033133""",30.0
…,…
"""722549575""",70.0
"""732997685""",30.0
"""716532384""",250.0


In [11]:
data = data.with_columns(
    pl.col('createdAt')
    .dt.hour()
    .alias('hour')
)

InvalidOperationError: `hour` operation not supported for dtype `str`

In [ ]:
data.group_by('hour').agg(pl.count().alias('Hourly_Rate'))

C:\Users\Nelly\AppData\Local\Temp\ipykernel_12272\2189232476.py:1: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  data.group_by('hour').agg(pl.count().alias('Hourly_Rate'))


hour,Hourly_Rate
i8,u32
19,87061


InvalidOperationError: `to_string` operation not supported for dtype `str`